In [2]:
#-----DATA UNDERSTANDING-----

#installation of libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt

#to display all columns and rows:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

#we determined how many numbers to show after comma
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('C:\\Users\Hakan\Downloads\marketing.csv', encoding = 'unicode_escape')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,3,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,3,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3,17850,United Kingdom


In [4]:
#ranking of the most ordered products
data.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending = False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039


In [5]:
#how many invoices are there in the data set
data["InvoiceNo"].nunique()

25900

In [6]:
#which are the most expensive products?
data.sort_values("UnitPrice", ascending = False).head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
222681,C556445,M,Manual,-1,6/10/2011 15:31,38970,15098,United Kingdom
524602,C580605,AMAZONFEE,AMAZON FEE,-1,12/5/2011 11:36,17836,nan,United Kingdom
43702,C540117,AMAZONFEE,AMAZON FEE,-1,1/5/2011 9:55,16888,nan,United Kingdom
43703,C540118,AMAZONFEE,AMAZON FEE,-1,1/5/2011 9:57,16454,nan,United Kingdom
15017,537632,AMAZONFEE,AMAZON FEE,1,12/7/2010 15:08,13541,nan,United Kingdom


In [7]:
#top 5 countries with the highest number of orders
data["Country"].value_counts().head()

United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Name: Country, dtype: int64

In [8]:
#total spending was added as a column
data['TotalPrice'] = data['UnitPrice']*data['Quantity']

In [9]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,3,17850,United Kingdom,15
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3,17850,United Kingdom,20
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,3,17850,United Kingdom,22
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3,17850,United Kingdom,20
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3,17850,United Kingdom,20


In [10]:
#-----DATA PREPARATION-----
data["InvoiceDate"].min() #oldest shopping date

'1/10/2011 10:04'

In [11]:
data["InvoiceDate"].max() #newest shopping date

'9/9/2011 9:52'

In [12]:
#to make the assessment easier, today's date is set as January 1, 2012.  
today = pd.datetime(2012,1,1) 
today

datetime.datetime(2012, 1, 1, 0, 0)

In [13]:
#changing the data type of the order date
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [14]:
#taking values greater than 0, this will be easier in terms of evaluation
data = data[data['Quantity'] > 0]
data = data[data['TotalPrice'] > 0]

In [15]:
data.shape #size information

(530104, 9)

In [16]:
data.describe([0.01,0.05,0.10,0.25,0.50,0.75,0.90,0.95, 0.99]).T
#explanatory statistics values of the observation units corresponding to the specified percentages
#processing according to numerical variables

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,530104,11,156,1,1,1,1,1,3,10,24,30,100,80995
UnitPrice,530104,4,36,0,0,0,1,1,2,4,8,10,17,13541
CustomerID,397884,15294,1713,12346,12415,12627,12883,13969,15159,16795,17725,17912,18211,18287
TotalPrice,530104,20,270,0,1,1,2,4,10,18,33,60,184,168470


In [17]:
#-----Finding RFM Score-----#

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530104 entries, 0 to 541908
Data columns (total 9 columns):
InvoiceNo      530104 non-null object
StockCode      530104 non-null object
Description    530104 non-null object
Quantity       530104 non-null int64
InvoiceDate    530104 non-null datetime64[ns]
UnitPrice      530104 non-null float64
CustomerID     397884 non-null float64
Country        530104 non-null object
TotalPrice     530104 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 40.4+ MB


In [19]:
# finding Recency and Monetary values.
data_x = data.groupby('CustomerID').agg({'TotalPrice': lambda x: x.sum(), #monetary value
                                        'InvoiceDate': lambda x: (today - x.max()).days}) #recency value
#x.max()).days; last shopping date of customers

In [20]:
data_y = data.groupby(['CustomerID','InvoiceDate']).agg({'TotalPrice': lambda x: x.sum()})
data_z = data_y.groupby('CustomerID').agg({'TotalPrice': lambda x: len(x)}) 
#finding the frequency value per capita

In [21]:
rfm_table= pd.merge(data_x,data_z, on='CustomerID')
#creating the RFM table

In [22]:
#determination of column names
rfm_table.rename(columns= {'InvoiceDate': 'Recency',
                          'TotalPrice_y': 'Frequency',
                          'TotalPrice_x': 'Monetary'}, inplace= True)

In [23]:
rfm_table.head()

,Monetary,Recency,Frequency
CustomerID,,,
12346,77184,347,1
12347,4310,24,7
12348,1797,97,4
12349,1758,40,1
12350,334,332,1


In [24]:
#RFM score values 
rfm_table['RecencyScore'] = pd.qcut(rfm_table['Recency'],5,labels=[5,4,3,2,1])
rfm_table['FrequencyScore'] = pd.qcut(rfm_table['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])
rfm_table['MonetaryScore'] = pd.qcut(rfm_table['Monetary'],5,labels=[1,2,3,4,5])

In [25]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore
CustomerID,,,,,,
12346,77184,347,1,1,1,5
12347,4310,24,7,5,5,5
12348,1797,97,4,2,4,4
12349,1758,40,1,4,1,4
12350,334,332,1,1,1,2


In [26]:
#RFM score values are combined side by side in str format
(rfm_table['RecencyScore'].astype(str) + 
 rfm_table['FrequencyScore'].astype(str) + 
 rfm_table['MonetaryScore'].astype(str)).head()

CustomerID
12346    115
12347    555
12348    244
12349    414
12350    112
dtype: object

In [27]:
#calculation of the RFM score
rfm_table["RFM_SCORE"] = rfm_table['RecencyScore'].astype(str) + rfm_table['FrequencyScore'].astype(str) + rfm_table['MonetaryScore'].astype(str)

In [28]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
CustomerID,,,,,,,
12346,77184,347,1,1,1,5,115
12347,4310,24,7,5,5,5,555
12348,1797,97,4,2,4,4,244
12349,1758,40,1,4,1,4,414
12350,334,332,1,1,1,2,112


In [29]:
#transposition of the RFM table. This makes it easier to evaluate.
rfm_table.describe().T

,count,mean,std,min,25%,50%,75%,max
Monetary,4338,2054,8989,4,307,674,1662,280206
Recency,4338,114,100,22,39,72,164,395
Frequency,4338,4,8,1,1,2,5,210


In [30]:
#customers with RFM Score 555
rfm_table[rfm_table["RFM_SCORE"] == "555"].head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
CustomerID,,,,,,,
12347,4310,24,7,5,5,5,555
12362,5226,25,10,5,5,5,555
12417,3649,25,9,5,5,5,555
12433,13376,22,7,5,5,5,555
12437,4951,23,18,5,5,5,555


In [31]:
#customers with RFM Score 111
rfm_table[rfm_table["RFM_SCORE"] == "111"].head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
CustomerID,,,,,,,
12353,89,226,1,1,1,1,111
12361,190,309,1,1,1,1,111
12401,84,325,1,1,1,1,111
12402,226,345,1,1,1,1,111
12441,174,388,1,1,1,1,111


In [32]:
#segmenting of customers according to RecencyScore and FrequencyScore values
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [33]:
#creation of segment variable
rfm_table['Segment'] = rfm_table['RecencyScore'].astype(str) + rfm_table['FrequencyScore'].astype(str)
rfm_table['Segment'] = rfm_table['Segment'].replace(seg_map, regex=True)

In [34]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
CustomerID,,,,,,,,
12346,77184,347,1,1,1,5,115,Hibernating
12347,4310,24,7,5,5,5,555,Champions
12348,1797,97,4,2,4,4,244,At Risk
12349,1758,40,1,4,1,4,414,Promising
12350,334,332,1,1,1,2,112,Hibernating


In [35]:
rfm_table[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])

Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
About to Sleep           74   353         1   353      473   353
At Risk                 175   594         3   594     1080   594
Can't Lose              153    64         8    64     2791    64
Champions                27   632        12   632     6867   632
Hibernating             239  1069         1  1069      489  1069
Loyal Customers          55   820         6   820     2863   820
Need Attention           73   184         2   184      894   184
New Customers            28    42         1    42      388    42
Potential Loyalists      38   486         2   486     1041   486
Promising                45    94         1    94      294    94